In [1]:
from dotenv import load_dotenv
import os

In [74]:
import json
from pathlib import Path
import re
import json
from openai import AzureOpenAI
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

load_dotenv()

azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_api_key = os.getenv("AZURE_OPENAI_API_KEY")
token_provider = get_bearer_token_provider(DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default")
azure_deployment = os.getenv("AZURE_OPENAI_DEPLOYMENT")

print(f"azure_endpoint: {azure_endpoint}")
print(f"azure_deployment: {azure_deployment}")

model_config = {
    "azure_endpoint": azure_endpoint,
    "azure_api_key": azure_api_key,
    "azure_deployment": azure_deployment,
}

client = AzureOpenAI(
        azure_endpoint=azure_endpoint,
        api_version=os.environ.get("AZURE_OPENAI_API_VERSION"),
        azure_ad_token_provider=token_provider,
        # api_key = api_key
        # azure_ad_token_provider=token_provider,
    )


azure_endpoint: https://azureaipriya6026657785.openai.azure.com/
azure_deployment: gpt-4o


In [3]:
from azure.ai.evaluation.simulator import Simulator

simulator = Simulator(model_config=model_config)

Class Simulator: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


In [90]:
def create_content_from_func_calls(func_calls):
    """
    Create a string content from a list of function call dictionaries.

    Args:
        func_calls (list): A list of dictionaries representing function calls. Each dictionary contains:
            - "role" (str): The role of the item, e.g., "assistant" or "function".
            - "function_call" (dict, optional): A dictionary with "name" and "arguments" keys, present if role is "assistant".
            - "content" (str, optional): The content of the assistant's response, present if role is "assistant".
            - "name" (str, optional): The name of the function, present if role is "function".

    Returns:
        str: A string representation of the function calls and responses, formatted with specific tags.
    """
        
    content = ""
    for item in func_calls:
        if item["role"] == "assistant" and "function_call" in item:
            content += "<functioncall> " + json.dumps({"name" : item['function_call']['name'], "arguments" : item['function_call']['arguments']}) + " "
        elif item["role"] == "assistant" and "content" in item:
            content += "<assistantresponse> " + item['content'] + " "
        elif item["role"] == "function":
            content += "<functionresponse> " + json.dumps({"name" : item['name'], "response" : item['content']}) + " "

    return content

def split_content(content, separators):
    """
    Split a string content into a list of dictionaries based on specified separators.

    Args:
        content (str): The string content to be split.
        separators (list): A list of separator strings to split the content.

    Returns:
        list: A list of dictionaries representing the split content. Each dictionary contains:
            - "role" (str): The role of the item, e.g., "assistant".
            - "function_call" (list, optional): A list of function call dictionaries, present if role is "assistant".
            - "name" (str, optional): The name of the function, present if role is "function".
            - "response" (str, optional): The response of the function, present if role is "function".
    """
    # Create a regex pattern to match any of the separators
    pattern = '|'.join(map(re.escape, separators))
    
    # Split the content based on the pattern
    split_result = re.split(f'({pattern})', content)
    
    # Filter out empty strings from the result
    split_result = [part for part in split_result if part]
    
    # Process the split result to create the expected output
    output = []
    i = 0
    while i < len(split_result):
        part = split_result[i]
        if part == "<functioncall>":
            while i < len(split_result) and split_result[i] == "<functioncall>":
                i += 1
                if i < len(split_result):
                    func_calls = json.loads(split_result[i])
                    output.append({"role": "assistant", "function_call": func_calls})
                i += 1
            i -= 1
        elif part == "<functionresponse>":
            while i < len(split_result) and split_result[i] == "<functionresponse>":
                i += 1
                if i < len(split_result):
                    func_response = json.loads(split_result[i])
                    func_name = func_response["name"]
                    content = func_response["response"]
                i += 1
                output.append({"role": "function", "name": func_name, "content": content})
            i -= 1
        elif part == "<assistantresponse>":
            output.append({"role": "assistant", "content": split_result[i + 1]})
            i += 1
        else:
            output.append(part)
        i += 1
    
    return output

In [1]:
from typing import List, Dict, Any, Optional
from functions import *
from function_list import function_list
from openai import AzureOpenAI
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

def call_to_ai_application(query: str) -> str:
    # logic to call your application
    # use a try except block to catch any errors
    system_message = "Assume the role of e-commerce assistant designed for multiple roles. You can help with creating promo codes, tracking their usage, checking stock levels, helping customers make shopping decisions and more. You have access to a bunch of tools that you can use to help you with your tasks. You can also ask the user for more information if needed."
    completion = client.chat.completions.create(
        model=azure_deployment,
        messages=[
            {"role" : "system",
             "content" : system_message
             },
            {
                "role": "user",
                "content": query,
            }
        ],
        max_tokens=800,
        temperature=0.1,
        top_p=0.2,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None,
        stream=False,
        tools = function_list,
        tool_choice="auto"
    )
    message = completion.choices[0].message
    # print("Message : ", message)
    # change this to return the response from your application
    return message

async def callback(
    messages: List[Dict],
    stream: bool = False,
    session_state: Any = None,  # noqa: ANN401
    context: Optional[Dict[str, Any]] = None,
) -> dict:
    messages_list = messages["messages"]
    # get last message
    latest_message = messages_list[-1]
    query = latest_message["content"]
    context = None
    # call your endpoint or ai application here
    response = call_to_ai_application(query)
    # we are formatting the response to follow the openAI chat protocol format:

    if response.tool_calls:
        prev_messages = messages["messages"]
        func_call_messages = []
        tool_calls = response.tool_calls

        ## Add the tool calls to the messages
        for tool_call in tool_calls:
            formatted_response = {"role" : "assistant", "function_call" : tool_call.function.to_dict()}
            func_call_messages.append(formatted_response)

        ## Execute the APIs and add the responses to the messages
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_args = tool_call.function.arguments
            func = globals().get(function_name)
            
            if callable(func):
                result = json.dumps(func(**json.loads(function_args)))
                # formatted_response = {"content" : result, "role" : "tool", "name" : function_name}
                formatted_response = {"role" : "function", "content" : result, "name" : function_name}
                func_call_messages.append(formatted_response)
            else:
                print("Function {} not found".format(function_name))   
 
        # Second API call: Get the final response from the model
        final_response = client.chat.completions.create(
            model=azure_deployment,
            messages=prev_messages + func_call_messages,
        )
        final_response = {"content" : final_response.choices[0].message.content, "role" : "assistant"}
        
        func_call_messages.append(final_response)
        # Stringify func_call messages to store in session state
        func_call_messages = create_content_from_func_calls(func_call_messages)
        func_call_messages = {"role" : "assistant", "content" : func_call_messages}
        
        messages["messages"].append(func_call_messages)
        # messages["messages"].append(final_response)

        return {"messages": messages["messages"], "stream": stream, "session_state": session_state}
    
    else:
        formatted_response = {
            "content": response.content,
            "role": "assistant",
        }
        messages["messages"].append(formatted_response)
        
    return {"messages": messages["messages"], "stream": stream, "session_state": session_state, "context": context}


In [30]:
current_directory = Path.cwd()
user_override_prompty = Path(current_directory, "user_override.prompty")
user_prompty_kwargs = {"mood" : "neutral"}

outputs = await simulator(
    target=callback,
    text="Assume the role of e-commerce assistant designed for multiple roles. You can help with creating promo codes, tracking their usage, checking stock levels, helping customers make shopping decisions and more. You have access to a bunch of tools that you can use to help you with your tasks. You can also ask the user for more information if needed.",
    num_queries=3,
    max_conversation_turns=5,
    tasks=[
        "I am a marketing manager. I want to create a promo code for our summer sale and track how many times it has been used so far?",
        "I am a customer and want to make a purchase using the promo code for the summer sale",
        "I am an inventory manager and want to check stock level for specific products",
    ],
    user_simulator_prompty=user_override_prompty,
    user_simulator_prompty_kwargs=user_prompty_kwargs,
)

Generating: 100%|██████████████████████████████████████████████| 15/15 [02:02<00:00,  8.15s/message]


In [31]:
outputs[-1]

{'messages': [{'role': 'user',
   'content': 'I would like to know the current stock levels for the following products: Product A, Product B, and Product C. Can you provide that information?',
   'context': 'None'},
  {'role': 'assistant',
   'content': 'Could you please provide me with the product IDs for Product A, Product B, and Product C? This will help me check the stock levels for you.',
   'context': ''},
  {'role': 'user',
   'content': 'Sure, here are the product IDs: Product A - ID123, Product B - ID456, Product C - ID789. Can you check the stock levels for these products?',
   'context': 'None'},
  {'role': 'assistant',
   'content': '<functioncall> {"name": "check_stock", "arguments": "{\\"product_id\\": \\"ID123\\"}"} <functioncall> {"name": "check_stock", "arguments": "{\\"product_id\\": \\"ID456\\"}"} <functioncall> {"name": "check_stock", "arguments": "{\\"product_id\\": \\"ID789\\"}"} <functionresponse> {"name": "check_stock", "response": "{\\"product_id\\": \\"ID123\\

In [87]:
final_outputs = []

for output in outputs:
    messages = []
    seperators = ["<functioncall>", "<assistantresponse>", "<functionresponse>"]
    for item in output["messages"]:
        if any(sep in item["content"] for sep in seperators):
            content = item["content"]
            content = split_content(content, seperators)
            messages.extend(content)
        else:
            messages.append({"role" : "assistant", "content" : item["content"]})

    final_outputs.append(messages)

In [88]:
final_outputs[0]

[{'role': 'assistant',
  'content': 'I am a marketing manager looking to create a promo code for our summer sale. Can you guide me on how to set it up and track its usage?'},
 {'role': 'assistant',
  'content': 'Certainly! Here\'s a step-by-step guide on how to create a promo code for your summer sale and track its usage:\n\n### Step 1: Create a Promo Code\n1. **Choose a Promo Code**: Decide on a unique code that customers will use at checkout. For example, "SUMMER2023".\n2. **Set the Discount**: Determine the discount percentage or amount you want to offer. For example, 20% off.\n3. **Set the Expiry Date**: Decide when the promo code will expire. For example, "2023-08-31".\n\n### Step 2: Track Promo Code Usage\nOnce the promo code is live, you can track its usage to see how many customers are using it and the impact on sales.\n\n### Let\'s Create the Promo Code\nPlease provide me with the following details so I can create the promo code for you:\n- The promo code you want to use.\n- T

In [89]:
# Save the outputs to a JSON file
with open("output.json", "w") as f:
    json.dump(final_outputs, f)